In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

PROJECT_JSON = kraft.read_json("../project.json")

PATH = make_path_dict(PROJECT_JSON)

In [ ]:
assert PROJECT_JSON["signal_type"] == "context"

In [ ]:
if "feature" in PROJECT_JSON["context_element_types"]:

    feature_context = pd.read_csv(
        PATH["feature_x_sample.feature_context.tsv"], sep="\t", index_col=0
    )

    feature_context.columns.name = PROJECT_JSON["sample_alias"]

    feature_context_negative = feature_context.clip(upper=0)

    feature_context_positive = feature_context.clip(lower=0)

In [ ]:
if "sample" in PROJECT_JSON["context_element_types"]:

    sample_context = pd.read_csv(
        PATH["feature_x_sample.sample_context.tsv"], sep="\t", index_col=0
    ).T

    sample_context.index.name = PROJECT_JSON["feature_alias"]

    sample_context_negative = sample_context.clip(upper=0)

    sample_context_positive = sample_context.clip(lower=0)

In [ ]:
if (
    "feature" in PROJECT_JSON["context_element_types"]
    and "sample" in PROJECT_JSON["context_element_types"]
):

    context_negative = kraft.normalize_dataframe(
        feature_context_negative, 1, "0-1"
    ) + kraft.normalize_dataframe(sample_context_negative, 0, "0-1")

    context_positive = kraft.normalize_dataframe(
        feature_context_positive, 1, "0-1"
    ) + kraft.normalize_dataframe(sample_context_positive, 0, "0-1")

elif "feature" in PROJECT_JSON["context_element_types"]:

    context_negative = feature_context_negative

    context_positive = feature_context_positive

elif "sample" in PROJECT_JSON["context_element_types"]:

    context_negative = sample_context_negative

    context_positive = sample_context_positive

In [ ]:
if PROJECT_JSON["select_feature_automatically"]:

    context_negative_features = kraft.select_series_indices(
        context_negative.sum(axis=1),
        "<",
        standard_deviation=PROJECT_JSON["select_feature_standard_deviation"],
        layout={
            "title": {"text": context_negative.index.name},
            "yaxis": {"title": "Negative Signal Sum"},
        },
    )

    context_positive_features = kraft.select_series_indices(
        context_positive.sum(axis=1),
        ">",
        standard_deviation=PROJECT_JSON["select_feature_standard_deviation"],
        layout={
            "title": {"text": context_negative.index.name},
            "yaxis": {"title": "Positive Signal Sum"},
        },
    )

else:

    context_negative_features = (
        context_negative.replace(0, np.nan).dropna(how="all").index
    )

    context_positive_features = (
        context_positive.replace(0, np.nan).dropna(how="all").index
    )

print(
    "Selected {} - & {} + feature.".format(
        context_negative_features.size, context_positive_features.size
    )
)

In [ ]:
if PROJECT_JSON["select_sample_automatically"]:

    context_negative_samples = kraft.select_series_indices(
        context_negative.sum(),
        "<",
        standard_deviation=PROJECT_JSON["select_sample_standard_deviation"],
        layout={
            "title": {"text": context_negative.columns.name},
            "yaxis": {"title": "Negative Signal Sum"},
        },
    )

    context_positive_samples = kraft.select_series_indices(
        context_positive.sum(),
        ">",
        standard_deviation=PROJECT_JSON["select_sample_standard_deviation"],
        layout={
            "title": {"text": context_negative.columns.name},
            "yaxis": {"title": "Positive Signal Sum"},
        },
    )

else:

    context_negative_samples = (
        context_negative.replace(0, np.nan).dropna(how="all").columns
    )

    context_positive_samples = (
        context_positive.replace(0, np.nan).dropna(how="all").columns
    )

print(
    "Selected {} - & {} + sample.".format(
        context_negative_samples.size, context_positive_samples.size
    )
)

In [ ]:
context_negative_signal = context_negative.loc[
    context_negative_features, context_negative_samples
]

context_positive_signal = context_positive.loc[
    context_positive_features, context_positive_samples
]

In [ ]:
if "negative" in PROJECT_JSON["contexts"] and "positive" in PROJECT_JSON["contexts"]:

    context_negative_signal.index = pd.Index(
        ("(-) {}".format(feature) for feature in context_negative_signal.index),
        name=context_negative_signal.index.name,
    )

    context_positive_signal.index = pd.Index(
        ("(+) {}".format(feature) for feature in context_positive_signal.index),
        name=context_positive_signal.index.name,
    )

    context_signal = pd.concat((-context_negative_signal, context_positive_signal))

elif "negative" in PROJECT_JSON["contexts"]:

    context_signal = -context_negative_signal

elif "positive" in PROJECT_JSON["contexts"]:

    context_signal = context_positive_signal

In [ ]:
if PROJECT_JSON["signal_normalization_method"] is not None:

    context_signal = kraft.normalize_dataframe(
        context_signal,
        PROJECT_JSON["signal_normalization_axis"],
        PROJECT_JSON["signal_normalization_method"],
    )

In [ ]:
context_signal.to_csv(PATH["feature_x_sample.signal.tsv"], sep="\t")

kraft.summarize_feature_x_sample(
    context_signal,
    feature_x_sample_alias="Context Signal",
    feature_x_sample_value_name="Context Signal",
    plot_heat_map_max_size=PROJECT_JSON["plot_heat_map_max_size"],
    plot_histogram_max_size=PROJECT_JSON["plot_histogram_max_size"],
)